<h1> Research iteration 2</h1>

<i>Joël Boafo, Sjoerd Beetsma, Maarten de Jeu
Class V2A - Group 5</i>

<h2>Business Understanding</h2>

Through the business, we have been asked to examine the following questions:

<ol>
<li>
With what accuracy can we decide the quality of a red-wine according to its chemical properties.
</li>
<li>Can we predict a wine's quality based on it's country origin</li>
<li>-.</li>
</ol>


<h2> Data Understanding </h2>

The dataset for the first research question is aquired from https://archive.ics.uci.edu/ml/datasets/wine+quality along a dataset about red-wine there is also a dataset about white-wine lets explore both for possible research-questions in the future.

The business tells us the variables in the chemical datasets are:<br />

1 - fixed acidity <br />
2 - volatile acidity <br />
3 - citric acid <br />
4 - residual sugar <br />
5 - chlorides <br />
6 - free sulfur dioxide <br />
7 - total sulfur dioxide <br />
8 - density <br />
9 - pH <br />
10 - sulphates <br />
11 - alcohol <br />
12 - quality (score between 0 and 10, based on sensory data) <br />

The business also let us now that they don't know if all variables are relevant in deciding the quality score of a wine.

For the second research question we aquired the dataset from ...
The business told us the dataset has the following variables:

1 - country<br/>
2 - description<br/>
3 - designation<br/>
4 - points<br/>
5 - price<br/>
6 - province <br/>
7 - region_1<br/>
8 - region_2<br/>
9 - taster_name<br/>
10 - taster_twitter_handle<br/>
11 - title <br/>
12 - variety<br/>
13 - winery<br/>

We import some libraries and the dataset to examine the data through code.

In [ ]:
import numpy as np
import pandas as pd
import sklearn as sk
from sklearn import preprocessing

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

Load in both red and white wine datasets

In [ ]:
dataset_red = pd.read_csv("datasets/winequality-red.csv", sep=";")
dataset_white = pd.read_csv("datasets/winequality-white.csv", sep=";")
dfReviews = pd.read_csv("datasets/winemag-data-130k-v2.csv")

A first look at the wine review dataset

In [ ]:
dfReviews.head(5)

At first glance we can see a wide range of countries and regions. It will be interesting to see if we can find a link between the assigned points of the wine, and where it's from.

Lets take the head of one of the chemical property datasets to have a first look at the submissions of the data.

In [ ]:
dataset_red.head()

As described by the business each row seems to correspond with a individual wine with eleven different columns describing chemical properties and one column which represents the quality score of the wine.

Lets see howmuch raw-data we have through the shapes of the datasets before we do any more exploration.

In [ ]:
red_rows, red_columns = dataset_red.shape
white_rows, white_columns = dataset_white.shape
reviews_rows, reviews_columns = dfReviews.shape

print(f'The red-wine quality dataset contains {red_rows} rows and {red_columns} columns')
print(f'The white-wine quality dataset contains {white_rows} rows and {white_columns} columns')
print(f'The wine-review dataset contains {reviews_rows} rows and {reviews_columns} columns')

Lets change the column name white spaces to underscores to make life easier.

In [ ]:
dataset_red.columns = dataset_red.columns.str.replace(' ','_')
dataset_white.columns = dataset_white.columns.str.replace(' ','_')

dataset_red.head(0)

<h3>Target and feature variables</h3>

All the columns describing chemical properties will be considerd as a feature variable and the column quality represents the target variable, the variable we want to predict.
Lets safe them in a variables for later use.

In [ ]:
feature_vars = ['fixed_acidity', 'volatile_acidity', 'citric_acid', 'residual_sugar', 'chlorides', 'free_sulfur_dioxide', 'total_sulfur_dioxide', 'density', 'pH', 'sulphates', 'alcohol'] 
target_var = 'quality'

All the columns describing the origin of the wine and the category will be considered as a feature variable. The column 'points' represents the target variable we want to predict.
We will also store them in variables

In [ ]:

feature_vars_reviews = ['country','designation', 'price','province','region_1','region_2','variety','winery']
target_var_reviews = ['points']
feature_target_vars_reviews = ['country','designation', 'price','province','region_1','region_2','variety','winery','points']

<h3> Scales of measurements </h3>

To choose a appropiate model for our research-questions and available data it's necessary to have a understanding of all the scales of measurements for the target and feature variables.

In [ ]:
nomi, disc, ordi, cont = 'Nominal', 'Discrete', 'Ordinal','Continous'
print('Scales of measurement chemical properties datasets')
pd.DataFrame(index=dataset_red.columns, data=[cont for i in range(11)] + [disc], columns=['Scale_of_measurement'])

As can be seen all the chemical properties (feature variables) have continous scale of measurement and the target variable, quality has a Discrete scale of measurement.

In [ ]:
var_scale = [nomi,nomi,cont,nomi,nomi,nomi,nomi,nomi,disc]
measurement_scales = pd.DataFrame(index=dfReviews[feature_target_vars_reviews].columns, data =var_scale, columns =['Scale_of_measurement'])
print('Scales of measurement wine-review dataset')
measurement_scales

Above we can see that most of the feature variables are nominal, with the exception bein price which is continious. Our target variable is discrete.

<h3>Central tendancies and dispersion measures</h3>

From the central tendancies and dispersion measures we can see some useful statistics about the target and feature variables.

In [ ]:
dataset_red.describe().round(2)

From the describe we can tell that there are quite a few columns with a big difference between maximum and minimum values which indicate outliers.

The columns with big differences between max and min values:
Residual_sugar, chlorides, free_sulfur_dioxide ,total_sulfur_dioxide.

In [ ]:
dataset_white.describe().round(2)

Just like the red-wine dataset, the white-wine dataset has similair differences in maximum and minimum values.

<h3>Distribution of data</h3>


Lets take a more visual look at the distribution of all data through a histogram for each of the feature and target attributes.

Starting off with the red wine dataset

In [ ]:
dataset_red.hist(figsize=(15,15))
plt.show()

Moving on to the white wine dataset:

In [ ]:
dataset_white.hist(figsize=(15,15))
plt.show()

As can been seen in the tables above the quality scores for the red wines range between 3 and 8 with wines with a score of 5 being the most common.
The white wines range within a quality of 3 and 9 with the score of 6 being most common.

<h3>Outliers</h3>

To get a visual understanding of the outliers in the feature columns each feature gets a boxplotted with the target variable points. Giving a small summary of the minimum, Q1, Q2 (median), Q3 and the maximum of each attribute plotted against points scored to give a view of outliers at all quality levels.

In [ ]:
def boxplotter(dataset, y_axes, x_axis):
    """Function to boxplot 1 x_axis against a list of y_axis of a given dataset"""
    for col in y_axes:
        sns.boxplot(x=dataset[x_axis], y=dataset[col])
        plt.show()

First boxplot all the feature variables against the target variable of the red-wine dataset

In [ ]:
boxplotter(dataset=dataset_red, y_axes=feature_vars, x_axis=target_var)

Now do the same for the white-wine dataset

In [ ]:
boxplotter(dataset=dataset_white, y_axes=feature_vars, x_axis=target_var)



As can be seen from the boxplots all of our current variables contain outliers.

All outliers in the above boxplots seem to be plausible and not from incorrect data.
From the boxplot with alcohol on the y axis and quality on the x axis we can  see that a trend of a rising median alcohol percentage the higher the quality of the wine.

<h3>Correlations</h3>

To help find correlations between variables and indepented/undepented attributes we can make use of a correlation matrix. 

In [ ]:
def corr_matrix_plotter(dataset, title=''):
    """Return a correlation matrix created using seaborn and matplotlib that for all columns in
    a pandas dataframe.

    Args:
        dataset: Dataset to construct correlation matrix for.
        title: Title of the plot.

    Returns:
        correlation matrix."""
    corr = dataset.corr()
    plt.figure(figsize=(10,7.5))
    cmap = sns.diverging_palette(200, 0, as_cmap=True) # color palette as cmap
    mask = np.logical_not(np.tril(np.ones_like(corr))) # triangle mask
    sns.heatmap(corr, annot=True, mask=mask, cmap = cmap, vmin=-1, vmax=1).set_title(title) # correlation heatmap
    plt.show()
corr_matrix_plotter(dataset_red, 'red-wine')
corr_matrix_plotter(dataset_white, 'white-wine')

In the correlation matrices graphed above you can see which attributes have a correlation to other attributes. Starting with our target variable 'quality', we can see quality has a few correlations with the strongest one being alcohol for both red and white wines and a few weaker ones like volatile acidity, sulphates and citric acid for red wines and density and chloride for white wines. Because quality is our target variable it's the indepented attribute in the correlations.

Besides there are some corelations among chemical properties:
Fixed acidity has strong correlation with pH, but it’s still an independent type. pH However is a dependent type; it depends on the former. Volatile acidity, residual sugar, sulphates, chlorides, and density are all independent data types. Total sulfur dioxide is dependent on free sulfur dioxide, but free sulfur dioxide is independent.

For the review dataset we check the direct correlation between price and point in a scatterplot. Since these are our only numeric values in de dataset.

In [ ]:
corr = dfReviews[feature_target_vars_reviews].corr()
plt.figure(figsize=(10,7.5))
sns.scatterplot(data = dfReviews, x='points', y='price')
plt.show()

In [ ]:
corr.round(2)

In the scatterplot above we can see some interesting findings. If we look at the price between the 500 and 100 it's very difficult to find a wine with points lower than 90. There are ofcourse plenty of wines that score 90 or higher that are below the 500 price mark, but the chance of finding one below 90 is also a lot higher. From this graph we find out that most of the time, the higher you go in price, the more sure you can be of the quality. an interesting finding to take into our model. The correlating number is 0.42 which means there certainly is some form of correlation.

<h3>Data Preparation</h3>

Lets start of the data preparation by checking the datatypes and clean or change them if necessary.

Red-wine quality datatypes:

In [ ]:
dataset_red.dtypes

White-wine quality datatypes:

In [ ]:
dataset_white.dtypes

The data-types of both red and white wine datasets have the same datatypes and each of the feature variables seem of the correct type.
Because we want to work with a classifier algorithm we will consider the quality as a categorical value since it also has few possible outcomes.

In [ ]:
dataset_red[[target_var]] = dataset_red[target_var].astype("category")
dataset_white[[target_var]] = dataset_white[target_var].astype("category")

We can now move on to checking and removing any NA values in the datasets.

In [ ]:
pd.isna(dataset_red).sum().sum() # checking for total NA values in red_wine

In [ ]:
pd.isna(dataset_white).sum().sum() # checking for total NA values in white_wine

Lets start of by removing all extreme the outliers leaving the mild ones in the dataset with a outer fence:

In [ ]:
def remove_outliers(dataset, fence = 3):
    q1 = dataset.quantile(.25)
    q3 = dataset.quantile(.75)
    iqr = q3 - q1
    return dataset[(dataset >= q1 - (fence * iqr)) & (dataset <= q3 + (fence * iqr))].dropna() # turn extreme outliers into NaN values and drop the rows

The red-wine dataset contained 1599 rows and the white-wine 4898 before removing the outliers lets remove the outliers and check howmany are left.

In [ ]:
dataset_red = remove_outliers(dataset_red)
dataset_white = remove_outliers(dataset_white)

In [ ]:
dataset_red.shape, dataset_white.shape

There are still 1435 rows left which means 12% of the columns contained outliers. With 88% still left there will be enough data to construct a model. 
There is roughly still 88% of the red-wine data left and 96% of the white-wine data after removing the extreme outliers that lay 3+ IQR above Q3 or 3+ IQR below Q1.

<h3>Normalizing data</h3>

Many algorithms used for making a prediction model work more efficient with normalized data. We can normalize the whole dataset into a new dataframe to acces normalized data from.

In [ ]:
def normalizer(dataset):
    scaler = sk.preprocessing.StandardScaler().fit(dataset)
    return pd.DataFrame((scaler.transform(dataset)), columns=dataset.columns)

In [ ]:
normalized_dataset_red = normalizer(dataset_red)
normalized_dataset_white = normalizer(dataset_white)

In [ ]:
normalized_dataset_red.head()

<h3>Data cleaned</h3>

The red and white wine chemicalproperty datasets got cleaned by removing all extreme outliers in the datasets and creating a normalized copy of both datasets.